In [1]:
%matplotlib widget

Check the current GPU usage. Please try to be nice!

In [2]:
!nvidia-smi

Sun Aug 26 11:16:41 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 396.44                 Driver Version: 396.44                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN V             On   | 00000000:03:00.0 Off |                  N/A |
| 28%   32C    P8    24W / 250W |      0MiB / 12066MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P100-PCIE...  On   | 00000000:83:00.0 Off |                    0 |
| N/A   33C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

> **WARNING**: The card numbers here are *not* the same as in CUDA. You have been warned.

## Imports

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import time
import torch

# Python 3 standard library
from pathlib import Path

### Set up local parameters

In [4]:
n_epochs = 100

# Name is the output file name
name = 'Aug_26_120000_SimpleCNN4Layer_C_100epochs_epsilon_1em3_lr_3em4'

# Make an output folder named "name" (change if you want)
output = Path(name)

# This is the input file to read in
datafile = Path('/data/schreihf/PvFinder/Aug_15_140000.npz')

# Size of batches
batch_size = 32

# How fast to learn
learning_rate = 3e-4

Make the output directory if it does not exist:

In [5]:
output.mkdir(exist_ok=True)

## Get the helper functions

Add the directory with the model
definitions to the path so we can import from it:

> When you type `import X`,
Python searches `sys.path` for a python
file named `X.py` to import. So we need to add the model directory to the path.

In [6]:
import sys
sys.path.append('../model')

In [8]:
# From model/collectdata.py
from collectdata import DataCollector

# From model/loss.py
from loss_epsilon_1em3 import Loss

# From model/training.py
from training import trainNet, select_gpu

# From model/models.py
from models_mds_C import SimpleCNN4Layer_C as Model

Set up Torch device configuration. All tensors and model parameters need to know where to be put.
This takes a BUS ID number: The BUS ID is the same as the listing at the top of this script.

In [9]:
device = select_gpu(2)

1 available GPUs (initially using device 0):
  0 TITAN V


## Loading data

Load the dataset, split into parts, then move to device (see `collectdata.py` in the `../model` directory)

In [10]:
collector = DataCollector(datafile, 120_000, 10_000)
train_loader = collector.get_training(batch_size, 120_000, device=device, shuffle=True)
val_loader = collector.get_validation(batch_size, 10_000, device=device, shuffle=False)

Loaded /data/schreihf/PvFinder/Aug_15_140000.npz in 9.337 s
Samples in Training: 120000 Validation: 10000 Test: 10000
Constructing dataset on cuda:0 took 14.35 s
Constructing dataset on cuda:0 took 0.4637 s


# Preparing the model

Prepare a model, use multiple GPUs if they are VISIBLE, and move the model to the device.

In [11]:
model = Model()
loss = Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [12]:
print("Let's use", torch.cuda.device_count(), "GPUs!")
if torch.cuda.device_count() > 1:
    model = torch.nn.DataParallel(model)

Let's use 1 GPUs!


Let's move the model's weight matricies to the GPU:

In [13]:
model = model.to(device)

## Train

The body of this loop runs once per epoch. Results is a named tuple of values (loss per epoch for training and validation, time each). Start by setting up a plot first:

In [14]:
fig, ax = plt.subplots()
lines_train, = ax.plot([], [], 'o-', label='Train')
lines_val, = ax.plot([], [], 'o-', label='Validation')
ax.set_xlabel('Epochs')
ax.set_ylabel('Cost')
plt.yscale('log') 
ax.legend();

FigureCanvasNbAgg()

In [15]:
# Run the epochs, using progress instead of range(n_epochs)
for results in trainNet(model, optimizer, loss,
                        train_loader, val_loader,
                        n_epochs,
                        notebook=True):
    
    # Update the plot above 
    lines_train.set_data(np.arange(len(results.cost)),results.cost)
    lines_val.set_data(np.arange(len(results.val)),results.val)
    
    
    #filter first cost epoch (can be really large)
    max_cost = max(max(results.cost if len(results.cost)<2 else results.cost[1:]),max(results.val))
    min_cost = min(min(results.cost),min(results.val))
    
    # The plot limits need updating too
    ax.set_ylim(min_cost*.9, max_cost*1.1)  
    ax.set_xlim(-.5, len(results.cost) - .5)
    
    # Redraw the figure
    fig.canvas.draw()

    # Save each model state dictionary
    torch.save(model.state_dict(), output / f'{name}_{results.epoch}.pyt')

Number of batches: train = 3750, val = 313


HBox(children=(IntProgress(value=0, description='Epochs', layout=Layout(flex='2')), HTML(value='')), layout=Layout(display='inline-flex', flex_flow='row wrap', width='100%'))

HBox(children=(IntProgress(value=0, description='Training', layout=Layout(flex='2'), max=3750), HTML(value='')), layout=Layout(display='inline-flex', flex_flow='row wrap', width='100%'))

Epoch 0: train=3.85666, val=1.0567, took 37.063 s
  Validation Found 0 of 55048, added 0 (eff 0.00%) (0.0 FP/event)


HBox(children=(IntProgress(value=0, description='Training', layout=Layout(flex='2'), max=3750), HTML(value='')), layout=Layout(display='inline-flex', flex_flow='row wrap', width='100%'))

...

Epoch 97: train=0.405313, val=0.418709, took 36.279 s
  Validation Found 40869 of 55048, added 1002 (eff 74.24%) (0.1 FP/event)


HBox(children=(IntProgress(value=0, description='Training', layout=Layout(flex='2'), max=3750), HTML(value='')), layout=Layout(display='inline-flex', flex_flow='row wrap', width='100%'))

Epoch 98: train=0.405333, val=0.42053, took 36.325 s
  Validation Found 40701 of 55048, added 1067 (eff 73.94%) (0.107 FP/event)


HBox(children=(IntProgress(value=0, description='Training', layout=Layout(flex='2'), max=3750), HTML(value='')), layout=Layout(display='inline-flex', flex_flow='row wrap', width='100%'))

Epoch 99: train=0.404884, val=0.419555, took 36.31 s
  Validation Found 40470 of 55048, added 1007 (eff 73.52%) (0.101 FP/event)



# Results

Let's save some results: (even though if you have not changed the code above, it saves the model every epoch)

Go ahead and save the final model (even though it was also saved above):

In [ ]:
torch.save(model.state_dict(), output / f'{name}_final.pyt')

Save the output results:

In [ ]:
np.savez(output / f'{name}_stats.npz', **results._asdict())

Save the plot above:

In [ ]:
fig.savefig(str(output / f'{name}_stats_a.png'))